In [1]:
# Установка nodejs и localtunnel
!curl -sL https://deb.nodesource.com/setup_14.x | bash -
!apt install -y nodejs
!npm install -g localtunnel

# Установка необходимых библиотек Python
!pip install flask pyngrok scikit-learn pandas



                              DEPRECATION WARNING                            

  Node.js 14.x is no longer actively supported!

  You will not receive security or critical stability updates for this version.

  You should migrate to a supported version of Node.js as soon as possible.
  Use the installation script that corresponds to the version of Node.js you
  wish to install. e.g.

   * https://deb.nodesource.com/setup_16.x — Node.js 16 "Gallium"
   * https://deb.nodesource.com/setup_18.x — Node.js 18 LTS "Hydrogen" (recommended)
   * https://deb.nodesource.com/setup_19.x — Node.js 19 "Nineteen"
   * https://deb.nodesource.com/setup_20.x — Node.js 20 "Iron" (current)

  Please see https://github.com/nodejs/Release for details about which
  version may be appropriate for you.

  The NodeSource Node.js distributions repository contains
  information both about supported versions of Node.js and supported Linux
  distributions. To learn more about usage, see the repository:
    https:/

In [7]:

import os
import joblib
import numpy as np
import threading
import subprocess
import time
import requests
from flask import Flask, render_template, request, send_from_directory
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Создаем необходимые папки
!mkdir -p templates static

# Создаем и обучаем модель (если файла нет)
def create_and_save_model():
    """Создает и сохраняет модель если файла нет"""
    iris = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Сохраняем модель с метаданными
    model_data = {
        'model': model,
        'target_names': iris.target_names.tolist(),
        'feature_names': ['Длина чашелистика', 'Ширина чашелистика', 'Длина лепестка', 'Ширина лепестка']
    }

    joblib.dump(model_data, 'iris_model_random_forest.pkl')
    print("✅ Модель создана и сохранена!")
    return model_data

# Загрузка или создание модели
try:
    model_data = joblib.load('iris_model_random_forest.pkl')
    print("✅ Модель загружена из файла!")
except:
    print("📦 Создаем новую модель...")
    model_data = create_and_save_model()

model = model_data['model']
target_names = model_data['target_names']
feature_names = model_data['feature_names']

# Создаем Flask приложение
app = Flask(__name__)

# HTML шаблоны
index_html = """
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Классификатор цветков Ириса</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
        .container { max-width: 800px; margin: 0 auto; background: white; border-radius: 15px; box-shadow: 0 20px 40px rgba(0,0,0,0.1); overflow: hidden; }
        .header { background: linear-gradient(135deg, #4CAF50, #45a049); color: white; padding: 30px; text-align: center; }
        .header h1 { font-size: 2.5em; margin-bottom: 10px; }
        .form-container { padding: 40px; }
        .form-group { margin-bottom: 25px; }
        .form-group label { display: block; margin-bottom: 8px; font-weight: bold; color: #333; }
        .form-group input { width: 100%; padding: 12px; border: 2px solid #ddd; border-radius: 8px; font-size: 1em; }
        .btn { background: linear-gradient(135deg, #4CAF50, #45a049); color: white; border: none; padding: 15px 30px; font-size: 1.1em; border-radius: 8px; cursor: pointer; width: 100%; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🌿 Классификатор Ирисов (Colab)</h1>
            <p>Определите вид цветка ириса по его параметрам</p>
        </div>

        <div class="form-container">
            <form action="/predict" method="POST">
                <div class="form-group">
                    <label for="feature1">Длина чашелистика (см):</label>
                    <input type="number" name="Длина чашелистика" step="0.1" min="4" max="8" required value="5.1">
                </div>
                <div class="form-group">
                    <label for="feature2">Ширина чашелистика (см):</label>
                    <input type="number" name="Ширина чашелистика" step="0.1" min="2" max="4.5" required value="3.5">
                </div>
                <div class="form-group">
                    <label for="feature3">Длина лепестка (см):</label>
                    <input type="number" name="Длина лепестка" step="0.1" min="1" max="7" required value="1.4">
                </div>
                <div class="form-group">
                    <label for="feature4">Ширина лепестка (см):</label>
                    <input type="number" name="Ширина лепестка" step="0.1" min="0.1" max="2.5" required value="0.2">
                </div>
                <button type="submit" class="btn">🔍 Определить вид Ириса</button>
            </form>
        </div>
    </div>
</body>
</html>
"""

result_html = """
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Результат классификации</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
        .container { max-width: 800px; margin: 0 auto; background: white; border-radius: 15px; box-shadow: 0 20px 40px rgba(0,0,0,0.1); overflow: hidden; }
        .header { background: linear-gradient(135deg, #4CAF50, #45a049); color: white; padding: 30px; text-align: center; }
        .content { padding: 40px; }
        .result-card { background: #f8f9fa; border-radius: 10px; padding: 30px; margin-bottom: 30px; text-align: center; }
        .species-name { font-size: 2.2em; color: #2c5530; margin: 20px 0; font-weight: bold; }
        .probability-chart { display: flex; justify-content: space-around; margin: 30px 0; flex-wrap: wrap; }
        .probability-item { flex: 1; min-width: 120px; margin: 10px; padding: 15px; border-radius: 10px; background: white; }
        .probability-item.max { background: #4CAF50; color: white; }
        .btn { display: inline-block; background: #4CAF50; color: white; text-decoration: none; padding: 15px 30px; border-radius: 8px; margin: 10px; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📊 Результат анализа</h1>
            <p>Вид цветка ириса определен</p>
        </div>

        <div class="content">
            {% if error %}
                <div style="background: #f8d7da; color: #721c24; padding: 20px; border-radius: 10px; text-align: center;">
                    <h2>❌ Ошибка</h2>
                    <p>{{ error }}</p>
                    <a href="/" class="btn">← Вернуться назад</a>
                </div>
            {% else %}
                <div class="result-card">
                    <h2>Определенный вид:</h2>
                    <div class="species-name">{{ class_name }}</div>
                    <div style="font-size: 1.3em; color: #4CAF50; margin-bottom: 20px;">Уверенность: {{ confidence }}%</div>

                    <div class="probability-chart">
                        {% for item in class_probabilities %}
                        <div class="probability-item {% if item.is_max %}max{% endif %}">
                            <div>{{ item.class_name }}</div>
                            <div style="font-size: 1.8em; font-weight: bold; margin: 10px 0;">{{ item.probability }}%</div>
                        </div>
                        {% endfor %}
                    </div>
                </div>

                <div style="text-align: center;">
                    <a href="/" class="btn">🔍 Новый анализ</a>
                </div>
            {% endif %}
        </div>
    </div>
</body>
</html>
"""

# Сохраняем HTML шаблоны
with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(index_html)

with open('templates/result.html', 'w', encoding='utf-8') as f:
    f.write(result_html)

# Маршруты Flask
@app.route('/')
def index():
    return index_html

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Получаем данные из формы
        features = [
            float(request.form['Длина чашелистика']),
            float(request.form['Ширина чашелистика']),
            float(request.form['Длина лепестка']),
            float(request.form['Ширина лепестка'])
        ]

        # Предсказание
        features_array = np.array([features])
        predicted_class = model.predict(features_array)[0]
        class_name = target_names[predicted_class]

        # Вероятности
        probabilities = model.predict_proba(features_array)[0]
        prob_percentages = [round(prob * 100, 2) for prob in probabilities]

        class_probabilities = []
        for i, prob in enumerate(prob_percentages):
            class_probabilities.append({
                'class_name': target_names[i],
                'probability': prob,
                'is_max': prob == max(prob_percentages)
            })

        # Русские названия
        class_translations = {
            'setosa': 'Сетоса (Ирис щетинистый)',
            'versicolor': 'Версиколор (Ирис разноцветный)',
            'virginica': 'Виргиника (Ирис виргинский)'
        }

        russian_class_name = class_translations.get(class_name.lower(), class_name)

        # Заменяем плейсхолдеры в HTML
        result_filled = result_html.replace('{{ class_name }}', russian_class_name)
        result_filled = result_filled.replace('{{ confidence }}', str(max(prob_percentages)))

        # Заменяем блок с вероятностями
        prob_html = ''
        for item in class_probabilities:
            prob_class = 'probability-item max' if item['is_max'] else 'probability-item'
            prob_html += f'''
            <div class="{prob_class}">
                <div>{item['class_name']}</div>
                <div style="font-size: 1.8em; font-weight: bold; margin: 10px 0;">{item['probability']}%</div>
            </div>
            '''

        result_filled = result_filled.replace('{% for item in class_probabilities %}', '')
        result_filled = result_filled.replace('{% endfor %}', '')
        result_filled = result_filled.replace('{% if item.is_max %}max{% endif %}', '')
        result_filled = result_filled.replace('{{ item.class_name }}', 'TEMP')
        result_filled = result_filled.replace('{{ item.probability }}', 'TEMP2')

        # Вставляем готовый HTML вероятностей
        start_marker = '<div class="probability-chart">'
        end_marker = '</div>'
        start_idx = result_filled.find(start_marker) + len(start_marker)
        end_idx = result_filled.find(end_marker, start_idx)

        result_filled = result_filled[:start_idx] + prob_html + result_filled[end_idx:]

        return result_filled

    except Exception as e:
        error_html = result_html.replace('{{ error }}', str(e))
        return error_html

# Функция для запуска Flask
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Функция для запуска LocalTunnel
def run_localtunnel():
    # Ждем немного, чтобы Flask успел запуститься
    time.sleep(3)

    # Получаем публичный IP для пароля
    try:
        ip_response = requests.get('https://api.ipify.org')
        public_ip = ip_response.text
        print(f"Пароль для доступа: {public_ip}")
    except:
        print("Не удалось получить IP автоматически")
        public_ip = "unknown"

    # Запускаем LocalTunnel с дополнительными параметрами
    process = subprocess.Popen(
        ['lt', '--port', '5000', '--print-requests', '--local-host', '127.0.0.1'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )

    # Читаем вывод для получения URL
    for line in process.stdout:
        if 'your url is:' in line:
            url = line.split('your url is: ')[1].strip()
            print(f"\n=== Ваш веб-сервис доступен по адресу: {url} ===")
            print(f"=== Пароль для доступа: {public_ip} ===")
            print("Скопируйте URL и откройте в браузере")
            print("При запросе пароля введите указанный выше IP-адрес")
            break

# Запускаем Flask в отдельном потоке
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Запускаем LocalTunnel
print("Запускаем LocalTunnel...")
run_localtunnel()

# Держим ячейку активной
while True:
    time.sleep(1)

✅ Модель загружена из файла!
Запускаем LocalTunnel...
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Пароль для доступа: 34.11.13.244

=== Ваш веб-сервис доступен по адресу: https://tired-queens-wink.loca.lt ===
=== Пароль для доступа: 34.11.13.244 ===
Скопируйте URL и откройте в браузере
При запросе пароля введите указанный выше IP-адрес


INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:22:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:22:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:22:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:22:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:22:47] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:23:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:23:35] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:23:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:27:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:30:59] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:31:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:31:13] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:31:32] "GET / H

KeyboardInterrupt: 